In [96]:
source_layer = "ra"
source_data = "2_14_Days_Ahead_Demand_Forecast"
source_container = "sink"
source_account = "dapffuksrawdl01"
source_file_path = "demand_forecasting/2_14_Days_Ahead_Demand_Forecast/CSV/NG_demand_14DA/2023/11/06"
#Printing the path
path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (source_container, source_account, source_file_path)
print(path)

StatementMeta(aspool, 273083, 162, Finished, Available)

abfss://sink@dapffuksrawdl01.dfs.core.windows.net/demand_forecasting/2_14_Days_Ahead_Demand_Forecast/CSV/NG_demand_14DA/2023/11/06


In [97]:
#Loading the file into a dataframe
Days_Ahead_Demand_Forecast_raw = spark.read.load('abfss://%s@%s.dfs.core.windows.net/%s' % (source_container, source_account, source_file_path), format='csv',Header= 'True')
print(Days_Ahead_Demand_Forecast_raw.count())
display(Days_Ahead_Demand_Forecast_raw)

StatementMeta(aspool, 273083, 163, Finished, Available)

143


SynapseWidget(Synapse.DataFrame, f438f70e-eec1-491f-8192-cadbc3a1d1b9)

In [98]:
from datetime import datetime
import pyspark.sql.types as T
import pyspark.sql.functions as F

def user_defined_timestamp(date_col):
    _date = datetime.strptime(date_col, '%Y%m%d')
    return _date.strftime('%Y-%m-%d')
user_defined_timestamp_udf = F.udf(user_defined_timestamp, T.StringType())
Days_Ahead_Demand_Forecast_raw1 = Days_Ahead_Demand_Forecast_raw.withColumn('TARGETDATE', user_defined_timestamp_udf('TARGETDATE'))
from pyspark.sql.functions import concat_ws
##write query for concatenate customi'd
Days_Ahead_Demand_Forecast_raw_reconc1=Days_Ahead_Demand_Forecast_raw1.withColumn("CUSTOM_ID", concat_ws("_","CP_ST_TIME","TARGETDATE"))
Days_Ahead_Demand_Forecast_raw_reconc1.printSchema()
display(Days_Ahead_Demand_Forecast_raw_reconc1)
print(Days_Ahead_Demand_Forecast_raw_reconc1.count())

StatementMeta(aspool, 273083, 164, Finished, Available)

root
 |-- DAYSAHEAD: string (nullable = true)
 |-- TARGETDATE: string (nullable = true)
 |-- FORECASTDEMAND: string (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: string (nullable = true)
 |-- CP_END_TIME: string (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)



SynapseWidget(Synapse.DataFrame, 46ba0d9f-b66a-4e4c-9dae-0c7205fbad19)

143


In [99]:
#List of parameters
target_layer = "cl"
target_source = "2to14days_demand_forecast"
target_container = "datatransformation"
target_account = "dapffukscleanseddl01"
target_file_path = "energy_balance/2to14days_demand_forecast/demand_cardinal_point/v1_deltalake"

StatementMeta(aspool, 273083, 165, Finished, Available)

In [100]:
days_demand_forecast_cleansed = spark.read.load('abfss://%s@%s.dfs.core.windows.net/%s/' % (target_container, target_account, target_file_path), format='delta')
print(days_demand_forecast_cleansed.count())
display(days_demand_forecast_cleansed)

StatementMeta(aspool, 273083, 166, Finished, Available)

1155


SynapseWidget(Synapse.DataFrame, af858c86-1305-41d6-b72f-512e3d1c1003)

In [101]:
days_demand_forecast_cleansed.printSchema()
Days_Ahead_Demand_Forecast_raw_reconc1.printSchema()

StatementMeta(aspool, 273083, 167, Finished, Available)

root
 |-- TARGETDATE: timestamp (nullable = false)
 |-- DAYSAHEAD: integer (nullable = false)
 |-- FORECASTDEMAND: integer (nullable = false)
 |-- CARDINALPOINT: string (nullable = false)
 |-- CP_TYPE: string (nullable = false)
 |-- CP_ST_TIME: integer (nullable = false)
 |-- CP_END_TIME: integer (nullable = false)
 |-- F_POINT: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)
 |-- _BATCHID: string (nullable = true)
 |-- _SRCFILENAME: string (nullable = true)
 |-- _CREATEDON: timestamp (nullable = true)
 |-- _UPDATEDON: timestamp (nullable = true)

root
 |-- DAYSAHEAD: string (nullable = true)
 |-- TARGETDATE: string (nullable = true)
 |-- FORECASTDEMAND: string (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: string (nullable = true)
 |-- CP_END_TIME: string (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)



In [102]:
%run /Tools/Utilities

StatementMeta(, , -1, Finished, Available)

..........Running Utilities Notebook.........


In [103]:
display(days_demand_forecast_cleansed.groupBy('_SRCFILENAME').count().where(col('count')>1))

StatementMeta(aspool, 273083, 182, Finished, Available)

SynapseWidget(Synapse.DataFrame, ea6321d0-0754-41b1-8d6e-b134a1f25d9b)

In [104]:
days_demand_forecast_cleansed_re=days_demand_forecast_cleansed.filter(days_demand_forecast_cleansed._SRCFILENAME=='/sink/demand_forecasting/2_14_Days_Ahead_Demand_Forecast/CSV/NG_demand_14DA/2023/11/06/NG_demand_14DA_20231120_202311061345_202311061345.csv')
print(days_demand_forecast_cleansed_re.count())
display(days_demand_forecast_cleansed_re)

StatementMeta(aspool, 273083, 183, Finished, Available)

143


SynapseWidget(Synapse.DataFrame, 72054638-2de7-472a-8fec-1c6e029026b9)

In [105]:
print(days_demand_forecast_cleansed_re.count())
print(days_demand_forecast_cleansed_re.dropDuplicates(["CUSTOM_ID"]).count())

StatementMeta(aspool, 273083, 184, Finished, Available)

143
143


In [106]:
print(days_demand_forecast_cleansed_re.count())
print(Days_Ahead_Demand_Forecast_raw_reconc1.count())

StatementMeta(aspool, 273083, 185, Finished, Available)

143
143


In [107]:
#Dropping some of the columns as they are externally added in Cleansed layer
cl_cols_to_drop = ["_BATCHID", "_SRCFILENAME", "_UPDATEDON", "_CREATEDON"]
days_demand_forecast_cleansed_reconc = days_demand_forecast_cleansed_re.drop(*cl_cols_to_drop)
#checking the RAW Layer data to see whether the column are dropped or not
display(days_demand_forecast_cleansed_reconc)


StatementMeta(aspool, 273083, 186, Finished, Available)

SynapseWidget(Synapse.DataFrame, 99e6317f-39da-445b-82c4-7886e7d9c069)

In [108]:
display(Days_Ahead_Demand_Forecast_raw_reconc1)
display(days_demand_forecast_cleansed_reconc)
display(Days_Ahead_Demand_Forecast_raw_reconc1.printSchema())
display(days_demand_forecast_cleansed_reconc.printSchema())

StatementMeta(aspool, 273083, 187, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7f3fbbf5-7b03-4677-bcd4-facc92cb6081)

SynapseWidget(Synapse.DataFrame, 28f3dc81-6105-4d2e-b19b-cc41de30c17e)

root
 |-- DAYSAHEAD: string (nullable = true)
 |-- TARGETDATE: string (nullable = true)
 |-- FORECASTDEMAND: string (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: string (nullable = true)
 |-- CP_END_TIME: string (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)

root
 |-- TARGETDATE: timestamp (nullable = false)
 |-- DAYSAHEAD: integer (nullable = false)
 |-- FORECASTDEMAND: integer (nullable = false)
 |-- CARDINALPOINT: string (nullable = false)
 |-- CP_TYPE: string (nullable = false)
 |-- CP_ST_TIME: integer (nullable = false)
 |-- CP_END_TIME: integer (nullable = false)
 |-- F_POINT: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)



In [109]:
#Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_n= Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new1[['CUSTOM_ID','SETTLEMENT_DATE','SETTLEMENT_PERIOD','ND','FORECAST_ACTUAL_INDICATOR','TSD','ENGLAND_WALES_DEMAND','EMBEDDED_WIND_GENERATION','EMBEDDED_WIND_CAPACITY','EMBEDDED_SOLAR_GENERATION','EMBEDDED_SOLAR_CAPACITY','NON_BM_STOR','PUMP_STORAGE_PUMPING','IFA_FLOW','IFA2_FLOW','BRITNED_FLOW','MOYLE_FLOW','EAST_WEST_FLOW','NEMO_FLOW','NSL_FLOW','ELECLINK_FLOW']]
#display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_n)

StatementMeta(aspool, 273083, 188, Finished, Available)

In [110]:
# changing the schema of columns from string to respective data types as mentioned in cleansed layer
from pyspark.sql.types import DateType,IntegerType,TimestampType
Days_Ahead_Demand_Forecast_raw_new = Days_Ahead_Demand_Forecast_raw_reconc1 \
            .withColumn("DAYSAHEAD" ,
              Days_Ahead_Demand_Forecast_raw_reconc1["DAYSAHEAD"]
              .cast(IntegerType()))  \
              .withColumn("TARGETDATE" ,
              Days_Ahead_Demand_Forecast_raw_reconc1["TARGETDATE"]
              .cast(TimestampType()))  \
            .withColumn("FORECASTDEMAND" ,
              Days_Ahead_Demand_Forecast_raw_reconc1["FORECASTDEMAND"]
              .cast(IntegerType()))  \
            .withColumn("CP_ST_TIME" ,
              Days_Ahead_Demand_Forecast_raw_reconc1["CP_ST_TIME"]
              .cast(IntegerType()))  \
            .withColumn("CP_END_TIME" ,
              Days_Ahead_Demand_Forecast_raw_reconc1["CP_END_TIME"]
              .cast(IntegerType()))  
            
Days_Ahead_Demand_Forecast_raw_new.printSchema()                              
display(Days_Ahead_Demand_Forecast_raw_new)


StatementMeta(aspool, 273083, 189, Finished, Available)

root
 |-- DAYSAHEAD: integer (nullable = true)
 |-- TARGETDATE: timestamp (nullable = true)
 |-- FORECASTDEMAND: integer (nullable = true)
 |-- CARDINALPOINT: string (nullable = true)
 |-- CP_TYPE: string (nullable = true)
 |-- CP_ST_TIME: integer (nullable = true)
 |-- CP_END_TIME: integer (nullable = true)
 |-- F_Point: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)



SynapseWidget(Synapse.DataFrame, c9768585-d2c2-4ec7-b254-9bf4737637c3)

In [111]:
Days_Ahead_Demand_Forecast_raw_reconc3= Days_Ahead_Demand_Forecast_raw_new[['TARGETDATE','DAYSAHEAD','FORECASTDEMAND','CARDINALPOINT','CP_TYPE','CP_ST_TIME','CP_END_TIME','F_POINT','CUSTOM_ID']]
display(Days_Ahead_Demand_Forecast_raw_reconc3)

StatementMeta(aspool, 273083, 190, Finished, Available)

SynapseWidget(Synapse.DataFrame, a34948e2-542a-4d39-9687-cdc781ff0c7f)

In [112]:
data_reconc = Days_Ahead_Demand_Forecast_raw_reconc3.exceptAll(days_demand_forecast_cleansed_reconc).unionAll(days_demand_forecast_cleansed_reconc.exceptAll(Days_Ahead_Demand_Forecast_raw_reconc3))
print(data_reconc.count())
display(data_reconc)

StatementMeta(aspool, 273083, 191, Finished, Available)

0


SynapseWidget(Synapse.DataFrame, 35771f38-7c10-48a7-a4fe-8249702cf8cc)

In [113]:
days_demand_forecast_cleansed_reconc.createOrReplaceTempView('Cleansed')

StatementMeta(aspool, 273083, 192, Finished, Available)

In [114]:
Days_Ahead_Demand_Forecast_raw_reconc3.createOrReplaceTempView('raw')

StatementMeta(aspool, 273083, 193, Finished, Available)

In [115]:
%%sql
select *,'Cl' from Cleansed where CUSTOM_ID='100_2023-10-29'
Union All
select *,'R' from raw where CUSTOM_ID='100_2023-10-29'

StatementMeta(aspool, 273083, 194, Finished, Available)

<Spark SQL result set with 0 rows and 10 fields>